In [ ]:
!pip install transformers datasets soundfile accelerate speechbrain==0.5.16

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
from datasets import load_dataset, Audio
dataset = load_dataset("erenfazlioglu/turkishvoicedataset", split="train")

In [28]:
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 130634
})

In [29]:
train_size = len(dataset) // 30
dataset = dataset.select(range(train_size))

print(dataset)


Dataset({
    features: ['audio', 'transcription'],
    num_rows: 4354
})


In [30]:

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [31]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
tokenizer = processor.tokenizer

In [33]:
dataset[:5]

{'audio': [{'path': '0000001.mp3',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000},
  {'path': '0000002.mp3',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000},
  {'path': '0000003.mp3',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000},
  {'path': '0000004.mp3',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000},
  {'path': '0000005.mp3',
   'array': array([0., 0., 0., ..., 0., 0., 0.]),
   'sampling_rate': 16000}],
 'transcription': ["Her geçen gün büyüyen LGBT sapkınlığına Instagram'dan bir destek daha geldi",
  "LGBT karşıtı içerikleri 'topluluk kurallarına aykırı' olduğu iddiası ile sansürleyen Instagram LGBT sapkınlığına olan desteğini bir kez daha gözler önüne serdi",
  'İBB Başkanı Ekrem İmamoğlu CHP Genel Başkanlığı için koltuk kavgasına girdi',
  'İstanbul aklında bile değildi',
  '4 yıldır ödeme alamayan özel halk otobüsü sahipleri isyan etti']}

In [34]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/4354 [00:00<?, ? examples/s]

In [35]:
dataset_vocab - tokenizer_vocab

{' ',
 '#',
 '%',
 '&',
 '+',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'Ç',
 'Ö',
 'Ü',
 'â',
 'ç',
 'î',
 'ö',
 'û',
 'ü',
 'Ğ',
 'ğ',
 'İ',
 'ı',
 'Ş',
 'ş',
 '̧',
 '–',
 '…'}

In [36]:
import re

def normalize_text(text):
    text = text.lower()

    text = re.sub(r'[^\w\s\']', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

def add_normalized_text(example):
    example['normalized_text'] = normalize_text(example['transcription'])
    return example


dataset = dataset.map(add_normalized_text)

print(dataset[2:5])

Map:   0%|          | 0/4354 [00:00<?, ? examples/s]

{'audio': [{'path': '0000003.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': '0000004.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}, {'path': '0000005.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 16000}], 'transcription': ['İBB Başkanı Ekrem İmamoğlu CHP Genel Başkanlığı için koltuk kavgasına girdi', 'İstanbul aklında bile değildi', '4 yıldır ödeme alamayan özel halk otobüsü sahipleri isyan etti'], 'normalized_text': ['ibb başkanı ekrem imamoğlu chp genel başkanlığı için koltuk kavgasına girdi', 'istanbul aklında bile değildi', '4 yıldır ödeme alamayan özel halk otobüsü sahipleri isyan etti']}


In [37]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

Map:   0%|          | 0/4354 [00:00<?, ? examples/s]

In [38]:
dataset_vocab - tokenizer_vocab

{' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'â',
 'ç',
 'î',
 'ö',
 'û',
 'ü',
 'ğ',
 'ı',
 'ş'}

In [39]:
replacements = [
    ("â", "a"),  
    ("ç", "ch"),  
    ("ğ", "gh"),  
    ("ı", "i"),   
    ("î", "i"),   
    ("ö", "oe"),  
    ("ş", "sh"),  
    ("ü", "ue"),  
    ("û", "u"),   
]

def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/4354 [00:00<?, ? examples/s]

In [40]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [41]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    example["labels"] = example["labels"][0]
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [42]:
processed_example = prepare_dataset(dataset[0])
list(processed_example.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [43]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/4354 [00:00<?, ? examples/s]

In [44]:
def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
len(dataset)

Filter:   0%|          | 0/4354 [00:00<?, ? examples/s]

4169

In [45]:
dataset = dataset.train_test_split(test_size=0.1)

In [46]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

       
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

       
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        

        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]


        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [47]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [48]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [49]:
from functools import partial

model.config.use_cache = False

model.generate = partial(model.generate, use_cache=True)

In [50]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./dharma_teja",  
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=500,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

In [18]:
trainer.train()

In [8]:
#testing
text = "Merhaba, ben Emirhan. Yapay zekayı çok seviyorum"

In [9]:

def cleanup_text(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [ ]:
cleaned_text = cleanup_text(text)
final_text = normalize_text(cleaned_text)
final_text

In [13]:
example = dataset["test"][304]

In [15]:
inputs = processor(text=final_text, return_tensors="pt")
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)

In [16]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [17]:
from IPython.display import Audio
import soundfile as sf

Audio(speech.numpy(), rate=16000)

sf.write('output.wav', speech.numpy(), 16000)